## <b><font color='darkblue'>What is runtime?</font></b>
<font size='3ptx'><b>The ADK Runtime is the underlying engine that powers your agent application during user interactions.</b> It's the system that takes your defined agents, tools, and callbacks and orchestrates their execution in response to user input, managing the flow of information, state changes, and interactions with external services like LLMs or storage.</font>

<b>Think of the Runtime as the "engine" of your agentic application.</b> You define the parts (<font color='brown'>agents, tools</font>), and the Runtime handles how they connect and run together to fulfill a user's request.

### <b><font color='darkgreen'>Core Idea: The Event Loop</font></b>
<font size='3ptx'><b>At its heart, the ADK Runtime operates on an Event Loop</b>. This loop facilitates a back-and-forth communication between the <b><font color='blue'>Runner</font></b> component and your defined "Execution Logic" (<font color='brown'>which includes your Agents, the LLM calls they make, Callbacks, and Tools</font>)</font>.

![event loop arch](https://google.github.io/adk-docs/assets/event-loop.png)

In simple terms:
1. <b>The `Runner`</b> receives a user query and asks the main `Agent` to start processing.
2. <b>The `Agent`</b> (<font color='brown'>and its associated logic</font>) runs until it has something to report (<font color='brown'>like a response, a request to use a tool, or a state change</font>) – it then yields or emits an `Event`.
3. <b>The `Runner` receives this Event</b>, processes any associated actions (<font color='brown'>like saving state changes via `Services`</font>), and forwards the event onwards (<font color='brown'>e.g., to the user interface</font>).
4. <b>Only after the `Runner` has processed the event does the `Agent`'s logic resume from where it paused</b>, now potentially seeing the effects of the changes committed by the `Runner`
5. <b>This cycle repeats until the agent has no more events to yield for the current user query</b>.

This event-driven loop is the fundamental pattern governing how ADK executes your agent code.

### <b><font color='darkgreen'>The Heartbeat: The Event Loop - Inner workings</font></b>
<font size='3ptx'><b>The Event Loop is the core operational pattern defining the interaction between the `Runner` and your custom code (<font color='brown'>`Agents`, `Tools`, `Callbacks`, collectively referred to as "Execution Logic" or "Logic Components" in the design document</font>)</b>. It establishes a clear division of responsibilities:</font>

#### <b>Runner's Role (Orchestrator)</b>
([source](https://google.github.io/adk-docs/runtime/#runners-role-orchestrator)) <b>The `Runner` acts as the central coordinator for a single user invocation</b>. Its responsibilities in the loop are:
1. <font size='3ptx'><b>Initiation</b></font>: Receives the end user's query (`new_message`) and typically appends it to the session history via the <b><font color='blue'>SessionService</font></b>.
2. <font size='3ptx'><b>Kick-off</font></b>: Starts the event generation process by calling the main agent's execution method (<font color='brown'>e.g., `agent_to_run.run_async(...)`</font>).
3. <font size='3ptx'><b>Receive & Process</b></font>: Waits for the agent logic to yield or emit an `Event`. Upon receiving an event, the `Runner` promptly processes it. This involves:
   - Using configured Services (`SessionService`, `ArtifactService`, `MemoryService`) to commit changes indicated in `event.actions` (<font color='brown'>like `state_delta`, `artifact_delta`</font>).
   - Performing other internal bookkeeping.
4. <font size='3ptx'><b>Yield Upstream</b></font>: Forwards the processed event onwards (<font color='brown'>e.g., to the calling application or UI for rendering</font>).
5. <font size='3ptx'><b>Iterate</b></font>: Signals the agent logic that processing is complete for the yielded event, allowing it to resume and generate the next event.

Below is the "Conceptual Runner Loop":

```python
# Simplified view of Runner's main loop logic
def run(new_query, ...) -> Generator[Event]:
    # 1. Append new_query to session event history (via SessionService)
    session_service.append_event(session, Event(author='user', content=new_query))

    # 2. Kick off event loop by calling the agent
    agent_event_generator = agent_to_run.run_async(context)

    async for event in agent_event_generator:
        # 3. Process the generated event and commit changes
        session_service.append_event(session, event) # Commits state/artifact deltas etc.
        # memory_service.update_memory(...) # If applicable
        # artifact_service might have already been called via context during agent run

        # 4. Yield event for upstream processing (e.g., UI rendering)
        yield event
        # Runner implicitly signals agent generator can continue after yielding
```

#### <b>Execution Logic's Role (Agent, Tool, Callback)</font>
([source](https://google.github.io/adk-docs/runtime/#execution-logics-role-agent-tool-callback)) <font size='3ptx'><b>Your code within agents, tools, and callbacks is responsible for the actual computation and decision-making</b></font>.

Its interaction with the loop involves:

1. <b><font size='3ptx'>Execute</font></b>: Runs its logic based on the current <b><font color='blue'>InvocationContext</font></b>, including the session state as it was when execution resumed.
2. <b><font size='3ptx'>Yield</font></b>: When the logic needs to communicate (<font color='brown'>send a message, call a tool, report a state change</font>), it constructs an `Event` containing the relevant content and actions, and then `yield`s this event back to the `Runner`.
3. <b><font size='3ptx'>Pause</font></b>: Crucially, execution of the agent logic pauses immediately after the `yield` statement. It waits for the `Runner` to complete step 3 (processing and committing).
4. <b><font size='3ptx'>Resume</font></b>: Only after the `Runner` has processed the yielded event does the agent logic resume execution from the statement immediately following the `yield`.
5. <b><font size='3ptx'>See Updated State</font></b>: Upon resumption, the agent logic can now reliably access the session state (`ctx.session.state`) reflecting the changes that were committed by the `Runner` from the previously yielded event.

Below is the "Conceptual Execution Logic:":
```python
# Simplified view of logic inside Agent.run_async, callbacks, or tools

# ... previous code runs based on current state ...

# 1. Determine a change or output is needed, construct the event
# Example: Updating state
update_data = {'field_1': 'value_2'}
event_with_state_change = Event(
    author=self.name,
    actions=EventActions(state_delta=update_data),
    content=types.Content(parts=[types.Part(text="State updated.")])
    # ... other event fields ...
)

# 2. Yield the event to the Runner for processing & commit
yield event_with_state_change
# <<<<<<<<<<<< EXECUTION PAUSES HERE >>>>>>>>>>>>

# <<<<<<<<<<<< RUNNER PROCESSES & COMMITS THE EVENT >>>>>>>>>>>>

# 3. Resume execution ONLY after Runner is done processing the above event.
# Now, the state committed by the Runner is reliably reflected.
# Subsequent code can safely assume the change from the yielded event happened.
val = ctx.session.state['field_1']
# here `val` is guaranteed to be "value_2" (assuming Runner committed successfully)
print(f"Resumed execution. Value of field_1 is now: {val}")

# ... subsequent code continues ...
# Maybe yield another event later...
```

This cooperative yield/pause/resume cycle between the `Runner` and your `Execution Logic`, mediated by `Event` objects, forms the core of the ADK Runtime.

### <b><font color='darkgreen'>Key components of the Runtime</font></b>
([source](https://google.github.io/adk-docs/runtime/#key-components-of-the-runtime)) <font size='3ptx'><b>Several components work together within the ADK Runtime to execute an agent invocation</b>. Understanding their roles clarifies how the event loop functions</font>:

1. <b><font size='3ptx'>Runner</font></b>
    - <b>Role</b>: The main entry point and orchestrator for a single user query (`run_async`).
    - <b>Function</b>: Manages the overall Event Loop, receives events yielded by the Execution Logic, coordinates with Services to process and commit event actions (<font color='brown'>state/artifact changes</font>), and forwards processed events upstream (<font color='brown'>e.g., to the UI</font>). It essentially drives the conversation turn by turn based on yielded events. (<font color='brown'>Defined in</font> <b><font color='blue'>google.adk.runners.runner</font></b>).

2. <b><font size='3ptx'>Execution Logic Components</font></b>
    - <b>Role:</b> The parts containing your custom code and the core agent capabilities.
    - <b>Components:</b>
        - `Agent` (`BaseAgent`, `LlmAgent`, etc.): Your primary logic units that process information and decide on actions. They implement the `_run_async_impl` method which yields events.
        - `Tools` (`BaseTool`, `FunctionTool`, `AgentTool`, etc.): External functions or capabilities used by agents (often `LlmAgent`) to interact with the outside world or perform specific tasks. They execute and return results, which are then wrapped in events.
        - `Callbacks` (`Functions`): User-defined functions attached to agents (e.g., `before_agent_callback`, `after_model_callback`) that hook into specific points in the execution flow, potentially modifying behavior or state, whose effects are captured in events.
    - <b>Function</b>: Perform the actual thinking, calculation, or external interaction. They communicate their results or needs by yielding `Event` objects and pausing until the `Runner` processes them.

3. <b><font size='3ptx'>Event</font></b>
    - <b>Role</b>: The message passed back and forth between the `Runner` and the `Execution Logic`.
    - <b>Function</b>: Represents an atomic occurrence (<font color='brown'>user input, agent text, tool call/result, state change request, control signal</font>). It carries both the content of the occurrence and the intended side effects (<font color='brown'>actions like `state_delta`</font>).

4. <b><font size='3ptx'>Services</font></b>
    - <b>Role</b>: Backend components responsible for managing persistent or shared resources. Used primarily by the `Runner` during event processing.
    - <b>Components</b>:
        - `SessionService` (`BaseSessionService`, `InMemorySessionService`, etc.): Manages `Session` objects, including saving/loading them, applying `state_delta` to the session state, and appending events to the `event history`.
        - `ArtifactService` (`BaseArtifactService`, `InMemoryArtifactService`, `GcsArtifactService`, etc.): Manages the storage and retrieval of binary artifact data. Although `save_artifact` is called via context during execution logic, the `artifact_delta` in the event confirms the action for the `Runner`/`SessionService`.
        - `MemoryService` (`BaseMemoryService`, etc.): (Optional) Manages long-term semantic memory across sessions for a user.
    - <b>Function</b>: Provide the persistence layer. The `Runner` interacts with them to ensure changes signaled by `event.actions` are reliably stored before the `Execution Logic` resumes.

5. <b><font size='3ptx'>Session</font></b>
    - <b>Role</b>: A data container holding the state and history for one specific conversation between a user and the application.
    - <b>Function</b>: Stores the `current state` dictionary, the list of all `past events` (`event history`), and references to associated artifacts. It's the primary record of the interaction, managed by the `SessionService`.

6. <b><font size='3ptx'>Invocation</font></b>
    - <b>Role</b>: A conceptual term representing everything that happens in response to a single user query, from the moment the `Runner` receives it until the agent logic finishes yielding events for that query.
    - <b>Function</b>: An invocation might involve multiple agent runs (<font color='brown'>if using agent transfer or `AgentTool`</font>), multiple LLM calls, tool executions, and callback executions, all tied together by a single `invocation_id` within the <b><font color='blue'>InvocationContext</font></b>.

These players interact continuously through the Event Loop to process a user's request.

### <b><font color='darkgreen'>How It Works: A Simplified Invocation</font></b>
([source](https://google.github.io/adk-docs/runtime/#how-it-works-a-simplified-invocation)) <b><font size='3ptx'>Let's trace a simplified flow for a typical user query that involves an LLM agent calling a tool:</font></b>
![flow](https://google.github.io/adk-docs/assets/invocation-flow.png)

#### <b>Step-by-Step Breakdown</b>
1. <b><font size='3ptx'>User Input</font></b>: The `User` sends a query (<font color='brown'>e.g., "What's the capital of France?"</font>).
2. <b><font size='3ptx'>Runner Starts</font></b>: `<b>Runner</b>.run_async` begins. It interacts with the <b><font color='blue'>SessionService</font></b> to load the relevant <b><font color='blue'>Session</font></b> and adds the user query as the first <b><font color='blue'>Event</font></b> to the `session history`. An <b><font color='blue'>InvocationContext</font></b> (`ctx`) is prepared.
3. <b><font size='3ptx'>Agent Execution</font></b>: The `Runner` calls `agent.run_async(ctx)` on the designated root agent (<font color='brown'>e.g., an</font> <b><font color='blue'>LlmAgent</font></b>).
4. <b><font size='3ptx'>LLM Call (Example)</font></b>: The `LlmAgent` determines if it needs information, perhaps by calling a tool. It prepares a request for the LLM. Let's assume the LLM decides to call `MyTool`.
5. <b><font size='3ptx'>Yield FunctionCall Event</font></b>: The `LlmAgent` receives the `FunctionCall` response from the LLM, wraps it in an `Event(author='Agent_Llm', content=Content(parts=[Part(function_call=...)]))`, and yields or emits this event.
6. <b><font size='3ptx'>Agent Pauses</font></b>: The `LlmAgent`'s execution pauses immediately after the yield.
7. <b><font size='3ptx'>Runner Processes</font></b>: The `Runner` receives the `FunctionCall` event. It passes it to the `SessionService` to record it in the history. The `Runner` then yields the event upstream to the `User` (<font color='brown'>or application</font>).
8. <b><font size='3ptx'>Agent Resumes</font></b>: The `Runner` signals that the event is processed, and `LlmAgent` resumes execution.
9. <b><font size='3ptx'>Tool Execution</font></b>: The `LlmAgent`'s internal flow now proceeds to execute the requested `MyTool`. It calls `tool.run_async(...)`.
10. <b><font size='3ptx'>Tool Returns Result</font></b>: `MyTool` executes and returns its result (e.g., `{'result': 'Paris'}`).
11. <b><font size='3ptx'>Yield FunctionResponse Event</font></b>: The `LlmAgent` wraps the tool result into an `Event` containing a `FunctionResponse` part (e.g., `Event(author='Agent_Llm', content=Content(role='user', parts=[Part(function_response=...)]))`). This event might also contain actions if the tool modified state (`state_delta`) or saved artifacts (`artifact_delta`). The agent yields this event.
12. <b><font size='3ptx'>Agent Pauses</font></b>: `LlmAgent` pauses again.
13. <b><font size='3ptx'>Runner Processes</font></b>: `Runner` receives the `FunctionResponse` event. It passes it to `SessionService` which applies any `state_delta`/`artifact_delta` and adds the event to history. `Runner` yields the event upstream.
14. <b><font size='3ptx'>Agent Resumes</font></b>: `LlmAgent` resumes, now knowing the tool result and any state changes are committed.
15. <b><font size='3ptx'>Final LLM Call (Example)</font></b>: `LlmAgent` sends the tool result back to the LLM to generate a natural language response.
16. <b><font size='3ptx'>Yield Final Text Event</font></b>: `LlmAgent` receives the final text from the LLM, wraps it in an `Event(author='Agent_Llm', content=Content(parts=[Part(text=...)]))`, and `yield`s it.
17. <b><font size='3ptx'>Agent Pauses</font></b>: `LlmAgent` pauses.
18. <b><font size='3ptx'>Runner Processes</font></b>: `Runner` receives the final text event, passes it to `SessionService` for history, and yields it upstream to the `User`. This is likely marked as the `is_final_response()`.
19. <b><font size='3ptx'>Agent Resumes & Finishes</font></b>: `LlmAgent` resumes. Having completed its task for this invocation, its `run_async` generator finishes.
20. <b><font size='3ptx'>Runner Completes</font></b>: The Runner sees the agent's generator is exhausted and finishes its loop for this invocation.

This yield/pause/process/resume cycle ensures that state changes are consistently applied and that the execution logic always operates on the most recently committed state after yielding an event.

### <b><font color='darkgreen'>Important Runtime Behaviors</font></b>
([source](https://google.github.io/adk-docs/runtime/#important-runtime-behaviors)) <b><font size='3ptx'>Understanding a few key aspects of how the ADK Runtime handles state, streaming, and asynchronous operations is crucial for building predictable and efficient agents.</font></b>

#### <b>State Updates & Commitment Timing</font>
* <b><font size='3ptx'>The Rule</font></b>: When your code (<font color='brown'>in an agent, tool, or callback</font>) modifies the session state (e.g., `context.state['my_key'] = 'new_value'`), this change is initially recorded locally within the current <b><font color='blue'>InvocationContext</font></b>. The change is <b>only guaranteed to be persisted</b> (<font color='brown'>saved by the `SessionService`</font>) after the `Event` carrying the corresponding `state_delta` in its actions has been `yield`-ed by your code and subsequently processed by the `Runner`.
* <b><font size='3ptx'>Implication</font></b>: Code that runs after resuming from a `yield` can reliably assume that the state changes signaled in the `yield`ed event have been committed.

```python
# Inside agent logic (conceptual)

# 1. Modify state
ctx.session.state['status'] = 'processing'
event1 = Event(..., actions=EventActions(state_delta={'status': 'processing'}))

# 2. Yield event with the delta
yield event1
# --- PAUSE --- Runner processes event1, SessionService commits 'status' = 'processing' ---

# 3. Resume execution
# Now it's safe to rely on the committed state
current_status = ctx.session.state['status'] # Guaranteed to be 'processing'
print(f"Status after resuming: {current_status}")
```

#### <b>"Dirty Reads" of Session State</b>
* <b><font size='3ptx'>Definition</font></b>: While commitment happens after the `yield`, code running later within the same invocation, but before the state-changing event is actually yielded and processed, can often see the local, uncommitted changes. This is sometimes called a "<b><font color='darkblue'>dirty read</font></b>".
* <b><font size='3ptx'>Example</font></b>:

```python
# Code in before_agent_callback
callback_context.state['field_1'] = 'value_1'
# State is locally set to 'value_1', but not yet committed by Runner

# ... agent runs ...

# Code in a tool called later *within the same invocation*
# Readable (dirty read), but 'value_1' isn't guaranteed persistent yet.
val = tool_context.state['field_1'] # 'val' will likely be 'value_1' here
print(f"Dirty read value in tool: {val}")

# Assume the event carrying the state_delta={'field_1': 'value_1'}
# is yielded *after* this tool runs and is processed by the Runner.
```

* <b><font size='3ptx'>Implications:</font></b>
    - <b>Benefit</b>: Allows different parts of your logic within a single complex step (e.g., multiple callbacks or tool calls before the next LLM turn) to coordinate using state without waiting for a full yield/commit cycle.
    - <b>Caveat</b>: Relying heavily on dirty reads for critical logic can be risky. <font color='darkred'>If the invocation fails before the event carrying the `state_delta` is yielded and processed by the `Runner`, the uncommitted state change will be lost</font>. For critical state transitions, ensure they are associated with an event that gets successfully processed.

#### <b>Streaming vs. Non-Streaming Output (`partial=True`)</b>
This primarily relates to how responses from the LLM are handled, especially when using streaming generation APIs.

* <b><font size='3ptx'>Streaming</font></b>: The LLM generates its response token-by-token or in small chunks.
    - <b>The framework</b> (often within `BaseLlmFlow`) <b>yields multiple `Event` objects for a single conceptual response</b>. Most of these events will have `partial=True`.
    - The `Runner`, upon receiving an event with `partial=True`, typically forwards it immediately upstream (<font color='brown'>for UI display</font>) but skips processing its actions (<font color='brown'>like `state_delta`</font>).
    - Eventually, the framework yields a final event for that response, marked as non-partial (`partial=False` or implicitly via `turn_complete=True`).
    - The `Runner` fully processes only this final event, committing any associated `state_delta` or `artifact_delta`.
* <b><font size='3ptx'>Non-Streaming</font></b>: The LLM generates the entire response at once. The framework yields a single event marked as non-partial, which the `Runner` processes fully.
* <b><font size='3ptx'>Why it Matters</font></b>: Ensures that state changes are applied atomically and only once based on the complete response from the LLM, while still allowing the UI to display text progressively as it's generated.

### <b><font color='darkgreen'>Async is Primary (`run_async`)</font></b>
* <b><font size='3ptx'>Core Design</font></b>: The ADK Runtime is fundamentally built on asynchronous libraries (<font color='brown'>Python's [**asyncio**](https://docs.python.org/3/library/asyncio.html)</font>) to handle concurrent operations (<font color='brown'>like waiting for LLM responses or tool executions</font>) efficiently without blocking.
* <b><font size='3ptx'>Main Entry Point</font></b>: `Runner.run_async` is the primary method for executing agent invocations. All core runnable components (<font color='brown'>Agents, specific flows</font>) use asynchronous methods internally.
* <b><font size='3ptx'>Synchronous Convenience (run)</font></b>: A synchronous `Runner.run` method exists mainly for convenience (<font color='brown'>e.g., in simple scripts or testing environments</font>). However, internally, `Runner.run` typically just calls `Runner.run_async` and manages the async event loop execution for you.
* <b><font size='3ptx'>Developer Experience</font></b>: We recommend designing your applications (<font color='brown'>e.g., web servers using ADK</font>) to be asynchronous for best performance. In Python, this means using [**asyncio**](https://docs.python.org/3/library/asyncio.html).
* <b><font size='3ptx'>Sync Callbacks/Tools</font></b>: The ADK framework supports both asynchronous and synchronous functions for tools and callbacks.
    - <b>Blocking I/O</b>: For long-running synchronous I/O operations, the framework attempts to prevent stalls. Python ADK may use [`asyncio.to_thread`](https://docs.python.org/3/library/asyncio-task.html#asyncio.to_thread).
    - <b>CPU-Bound Work</b>: Purely CPU-intensive synchronous tasks will still block their execution thread in both environments.

Understanding these behaviors helps you write more robust ADK applications and debug issues related to state consistency, streaming updates, and asynchronous execution.